<a href="https://colab.research.google.com/github/AsmaaYassinDev/Behavioural-Anomaly-Detection-for-ATO-Fraud/blob/main/Unsupervised_Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
import warnings

# Ignore unimportant warnings
warnings.filterwarnings('ignore', category=UserWarning)

print("--- Execution Started (Stable Code) ---")
print("Objective: Identify Anomalous Accounts based on their behavior.")

# --- Load Data (Make sure this name matches the file you uploaded) ---
file_path = 'PS_20174392719_1491204439457_log24.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded the full file ({len(df)} rows).")
except Exception as e:
    print(f"Error during data loading: {e}")
    print("!!! Make sure the file name in the code (file_path) matches the file in the folder exactly !!!")
    exit()

# --- Step 1: Create the 'Smart Sample' ---
print("\n--- Step 1: Creating the 'Smart Sample' ---")
df_fraud = df[df['isFraud'] == 1]
fraud_dest_ids = df_fraud['nameDest'].unique()
fraud_orig_ids = df_fraud['nameOrig'].unique()
all_fraud_user_ids = np.union1d(fraud_dest_ids, fraud_orig_ids)
df_fraud_lifecycle = df[
    df['nameOrig'].isin(all_fraud_user_ids) |
    df['nameDest'].isin(all_fraud_user_ids)
]

df_normal = df[df['isFraud'] == 0]
sample_size = min(500000, len(df_normal))
df_normal_sample = df_normal.sample(n=sample_size, random_state=42)

df_smart_sample = pd.concat([df_fraud_lifecycle, df_normal_sample]).drop_duplicates(keep='first')
print(f"The final 'Smart Sample' was created with {len(df_smart_sample)} rows.")

# --- Step 2: Build Behavioral Profiles (Strong Features) ---
print("\n--- Step 2: Building Behavioral Profiles ---")

# (a) Calculate total received, total cashed out, and unique senders count
df_received = df_smart_sample[df_smart_sample['type'].isin(['TRANSFER', 'CASH_IN'])]
total_received = df_received.groupby('nameDest')['amount'].sum().to_dict()
unique_senders = df_received.groupby('nameDest')['nameOrig'].nunique().to_dict()

df_cashed_out = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT']
total_cashed_out = df_cashed_out.groupby('nameOrig')['amount'].sum().to_dict()

all_user_ids = set(total_received.keys()) | set(total_cashed_out.keys()) | set(unique_senders.keys())
profiles_list = []
for user_id in all_user_ids:
    received = total_received.get(user_id, 0)
    cashed_out = total_cashed_out.get(user_id, 0)
    senders = unique_senders.get(user_id, 0)

    ratio = (cashed_out / (received + 1e-6))
    ratio = min(ratio, 1.0) # The ratio cannot exceed 100%

    profiles_list.append({
        'user_id': user_id,
        'dest_cash_out_ratio': ratio,
        'dest_unique_senders': senders
    })

final_profiles = pd.DataFrame(profiles_list)
print("Behavioral profiles created successfully.")

# --- Step 3: Merge Features with Transactions ---
print("\n--- Step 3: Merging Features with Transactions ---")
df_model_data = pd.merge(df_smart_sample, final_profiles, left_on='nameDest', right_on='user_id', how='left')
df_model_data = pd.merge(df_model_data, final_profiles, left_on='nameOrig', right_on='user_id', how='left', suffixes=('_dest', '_orig'))

df_model_data['dest_cash_out_ratio_dest'] = df_model_data['dest_cash_out_ratio_dest'].fillna(0)
df_model_data['dest_unique_senders_dest'] = df_model_data['dest_unique_senders_dest'].fillna(0)
df_model_data['dest_cash_out_ratio_orig'] = df_model_data['dest_cash_out_ratio_orig'].fillna(0)
df_model_data['dest_unique_senders_orig'] = df_model_data['dest_unique_senders_orig'].fillna(0)

# --- Step 4: Train an Unsupervised Model ---
print("\n--- Step 4: Training Isolation Forest Model ---")

features = [
    'amount',
    'dest_cash_out_ratio_dest', # Recipient's cash-out ratio
    'dest_unique_senders_dest', # Recipient's unique senders
    'dest_cash_out_ratio_orig', # Sender's cash-out ratio
    'dest_unique_senders_orig'  # Sender's unique senders
]
df_model_data['type_encoded'] = df_model_data['type'].astype('category').cat.codes
features.append('type_encoded')

X = df_model_data[features]
y_true = df_model_data['isFraud'] # The "Correct Answer"

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

contamination = y_true.mean()
print(f"Fraud (Contamination) rate in the Smart Sample: {contamination:.2%}")

model = IsolationForest(contamination=contamination, random_state=42)
model.fit(X_scaled)
predictions = model.predict(X_scaled) # -1 = anomalous, 1 = normal
print("Model training complete.")

# --- Step 5: Answer Your Question ---
print("\n--- Step 5: Identifying Anomalous Accounts ---")

# Add the model's "guess" to the data
# 1 = anomalous, 0 = normal
df_model_data['anomaly_prediction'] = [1 if p == -1 else 0 for p in predictions]

# Find the "transactions" that the model judged as anomalous
anomalous_transactions = df_model_data[df_model_data['anomaly_prediction'] == 1]

# Find the "account names" (senders and recipients) involved in these anomalous transactions
anomalous_dest_accounts = anomalous_transactions['nameDest'].unique()
anomalous_orig_accounts = anomalous_transactions['nameOrig'].unique()

all_anomalous_accounts = np.union1d(anomalous_dest_accounts, anomalous_orig_accounts)

print(f"\n[Final Result]: The model found {len(all_anomalous_accounts)} 'anomalous' accounts.")

# Print a sample of 20 accounts the model considered 'anomalous'
print("Sample of accounts the model considered 'anomalous':")
print(all_anomalous_accounts[:20])

# --- For Comparison: What are the "Real" Fraudulent Accounts? ---
print("\n--- For Comparison (The Truth) ---")
print(f"The number of 'real' fraudulent accounts (isFraud=1) was: {len(all_fraud_user_ids)}")
print("Sample of 'real' fraudulent accounts:")
print(all_fraud_user_ids[:20])

# Calculate F1-Score to verify quality
f1 = f1_score(y_true, df_model_data['anomaly_prediction'])
print(f"\nThe model's F1-Score (for confirmation): {f1:.2%}")

print("\n--- Stable Code Execution Complete ---")

--- Execution Started (Stable Code) ---
Objective: Identify Anomalous Accounts based on their behavior.
Successfully loaded the full file (1048575 rows).

--- Step 1: Creating the 'Smart Sample' ---
The final 'Smart Sample' was created with 504964 rows.

--- Step 2: Building Behavioral Profiles ---
Behavioral profiles created successfully.

--- Step 3: Merging Features with Transactions ---

--- Step 4: Training Isolation Forest Model ---
Fraud (Contamination) rate in the Smart Sample: 0.23%
Model training complete.

--- Step 5: Identifying Anomalous Accounts ---

[Final Result]: The model found 1950 'anomalous' accounts.
Sample of accounts the model considered 'anomalous':
['C1000820773' 'C1000839468' 'C1001444586' 'C10015111' 'C1003526443'
 'C1003775387' 'C1007717381' 'C1008947638' 'C1010765614' 'C1011097249'
 'C1013511446' 'C1013700132' 'C1014154376' 'C1015888357' 'C1016521533'
 'C1017653240' 'C1018394275' 'C1021713645' 'C102174220' 'C1022269511']

--- For Comparison (The Truth) ---